In [23]:
using SparseArrays, LinearAlgebra

"""
    sparse_solve!(X, K, F)

Megoldja a `K * X = F` egyenletet, ahol `K` ritka,
`F` és `X` is ritka `SparseMatrixCSC`.
A `K` egyszeri LU-faktorizációját végzi el, majd minden oszlopot külön old meg.
Az eredményt `X`-be írja (ha nem üres).
"""
function sparse_solve!(X::SparseMatrixCSC, K::SparseMatrixCSC, F::SparseMatrixCSC)
    n, m = size(K, 1), size(F, 2)
    Ffac = lu(K)                       # egyszeri faktorozás
    x = zeros(n)                       # ideiglenes sűrű megoldás
    b = zeros(n)                       # ideiglenes jobboldal
    I, V, J = Int[], Float64[], Int[]  # ritka kitöltéshez

    for j in 1:m
        # sparse -> dense jobboldal
        idx, val = findnz(view(F, :, j))
        b .= 0
        b[idx] .= val

        # megoldás (helyben, allokáció nélkül)
        ldiv!(x, Ffac, b)

        # dense -> sparse visszatöltés
        nz = findall(!iszero, x)
        append!(I, nz)
        append!(J, fill(j, length(nz)))
        append!(V, x[nz])
    end

    X = sparse(I, J, V, n, m)          # új sparse mátrix
    return X
end

"""
    sparse_solve(K, F) -> X

Ugyanaz, mint `sparse_solve!`, de automatikusan létrehozza `X`-et.
"""
function sparse_solve(K::SparseMatrixCSC, F::SparseMatrixCSC)
    X = spzeros(size(K, 1), size(F, 2))
    return sparse_solve!(X, K, F)
end


sparse_solve

In [24]:
K = sprand(5000, 5000, 0.002) + I
F = sprand(1000, 100, 0.002)

@time X = sparse_solve(K, F)


  2.915017 seconds (256.41 k allocations: 495.663 MiB, 0.25% gc time, 8.14% compilation time)


5000×100 SparseMatrixCSC{Float64, Int64} with 434913 stored entries:
⎡⣿⣿⎤
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎢⣿⣿⎥
⎣⣿⣿⎦

In [25]:
using SparseArrays, LinearAlgebra, BenchmarkTools

# ========== TESZTADATOK ==========
n = 5000
densK = 0.002      # K sűrűsége
densF = 0.002       # F sűrűsége
m = 100             # jobboldalak száma (F oszlopok)

K = sprand(n, n, densK) + I
F_sparse = sprand(n, m, densF)
F_dense = Matrix(F_sparse)             # referencia jobboldal

# ========== MODERN SPARSE SOLVER ==========
function sparse_solve2!(X::SparseMatrixCSC, K::SparseMatrixCSC, F::SparseMatrixCSC)
    n, m = size(K, 1), size(F, 2)
    Ffac = lu(K)
    x = zeros(n)
    b = zeros(n)
    I, V, J = Int[], Float64[], Int[]

    for j in 1:m
        idx, val = findnz(view(F, :, j))
        b .= 0
        b[idx] .= val
        ldiv!(x, Ffac, b)
        nz = findall(!iszero, x)
        append!(I, nz)
        append!(J, fill(j, length(nz)))
        append!(V, x[nz])
    end

    X = sparse(I, J, V, n, m)
    return X
end

function sparse_solve2(K::SparseMatrixCSC, F::SparseMatrixCSC)
    X = spzeros(size(K, 1), size(F, 2))
    return sparse_solve2!(X, K, F)
end

# ========== BENCHMARKOK ==========
println("🧮 Méret: $(n)x$(n), $(m) jobboldal, densK=$(densK), densF=$(densF)")

@btime X1 = sparse_solve($K, $F_sparse);
@btime X2 = sparse_solve2($K, $F_sparse);
@btime X3 = $K \ $F_dense;

# ========== ELLENŐRZÉS ==========
X1 = sparse_solve(K, F_sparse);
X2 = sparse_solve2(K, F_sparse);
X3 = K \ F_dense;
@assert norm(Matrix(X1) - X3) / norm(X2) < 1e-10
@assert norm(Matrix(X2) - X3) / norm(X2) < 1e-10
println("✅ Megoldások egyeznek numerikusan.")


🧮 Méret: 5000x5000, 100 jobboldal, densK=0.002, densF=0.002
  2.640 s (1660 allocations: 271.44 MiB)
  2.726 s (1660 allocations: 271.44 MiB)
  2.631 s (93 allocations: 225.98 MiB)
✅ Megoldások egyeznek numerikusan.
